Import Libraries

In [1]:
from itertools import compress
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import interpolate
import math
import optuna

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut, GridSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.covariance import LedoitWolf
from sklearn.utils import resample
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from mne.decoding import UnsupervisedSpatialFilter, SlidingEstimator
from mne.preprocessing.nirs import optical_density, beer_lambert_law, scalp_coupling_index, temporal_derivative_distribution_repair
from mne_nirs.channels import get_long_channels, get_short_channels
from mne_nirs.channels import picks_pair_to_idx as p2idx
from mne_nirs.signal_enhancement import (enhance_negative_correlation,
                                         short_channel_regression)
from mne.viz import plot_compare_evokeds
from mne_nirs.experimental_design import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix

import mne
from mne.decoding import Scaler, cross_val_multiscore, Vectorizer
from mne import Epochs, events_from_annotations, set_log_level
from mne.io import read_raw_snirf
from mne_nirs.io.snirf import write_raw_snirf
from mne_bids import write_raw_bids, BIDSPath, read_raw_bids

# Feature extraction
from tsfresh import extract_features, extract_relevant_features, select_features, feature_extraction
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.feature_calculators import set_property
from tsfresh.feature_extraction import feature_calculators

# Feature importance
from eli5.sklearn import PermutationImportance

import h5py as h 

In [2]:
def individual_analysis(bids_path):
    # Read data with annotations in BIDS format
    raw_intensity = read_raw_bids(bids_path, verbose=False)

    # Convert signal to optical density and determine bad channels
    raw_od = optical_density(raw_intensity)
    sci = scalp_coupling_index(raw_od, h_freq=1.35, h_trans_bandwidth=0.1)
    raw_od.info["bads"] = list(compress(raw_od.ch_names, sci < 0.5))
    raw_od.interpolate_bads()

    # Downsample and apply signal cleaning techniques
    raw_od.resample(1.0)
    raw_od = temporal_derivative_distribution_repair(raw_od)
    raw_od = short_channel_regression(raw_od)

    # Convert to haemoglobin and filter
    raw_haemo = beer_lambert_law(raw_od)
    
    ## FILTERS OUT HEART RATE
    raw_haemo = raw_haemo.filter(None, 0.4,
                                 h_trans_bandwidth=0.1, l_trans_bandwidth=0.01,
                                 verbose=False)
    raw_haemo.annotations.delete(raw_haemo.annotations.description == '15')
    
    # Apply further data cleaning techniques and extract epochs
    raw_haemo = enhance_negative_correlation(raw_haemo)

    # raw_haemo = get_long_channels(raw_haemo, min_dist=0.01)

    # Pick data channels that are actually informative
    # roi_channels = mne.pick_channels(raw_haemo.info['ch_names'], include=['Left_PT','Right_PT'])
    # raw_haemo = raw_haemo.copy().pick_channels(roi_channels)

    # Extract events but ignore those with
    events, event_dict = events_from_annotations(raw_haemo, verbose=False,
                                                 regexp='^(?![Ends]).*$')
    
    epochs = Epochs(raw_haemo, events, event_id=event_dict, tmin=-5, tmax=30,
                    reject=dict(hbo=100e-6), reject_by_annotation=True,
                    proj=True, baseline=(None, 0), detrend=1,
                    preload=True, verbose=False,event_repeated='merge')

    return raw_haemo, epochs

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb




def get_id(time):
    time_in_stims = stims['time']
    matches = stims[(time_in_stims >= time - 30) & (time_in_stims <= time + 5)]
    if len(matches) > 0:
        # print(f"\nmatches: \n{matches}\n")
        return matches.iloc[0]['id']
    else:
        return None


# for ch_type in ["fnirs", "hbo", "hbr"]:
for ch_type in ["hbo"]:
    x_all = []
    y_all = []
    groups = []

    for sub in range(1,2):
        # hbo = pd.DataFrame()
        # target = pd.Series()
        
        subject_id = "%02d" % sub
        raw_path = BIDSPath(
            subject="%02d" % sub,
            # task="task",
            session='01',
            datatype="nirs",
            # suffix='nirs',
            root=r"C:\Users\dalto\Downloads\project\sourcedata_stim",
            extension=".snirf"
        )

        # events, event_id = mne.events_from_annotations(mne.io.read_raw_snirf(raw_path))
        # raw_haemo, epochs = individual_analysis(raw_path)

        # print(f"epochs: \n{epochs.events[37:, [0, 2]]}\n")
        # print(f"epochs size: {raw_haemo.info}\n")

        raw_haemo = read_raw_bids(raw_path)

        hbo_cols = raw_haemo.info.ch_names[::2]
        print(f"raw.info: {len(hbo_cols)}\n")
        

        with h.File(raw_path,'r') as f:
            haemo_multi = f['nirs/data1']
            x_list = list(haemo_multi)
            print(f"x available lists: {x_list}\n")

            x_data = haemo_multi.get('dataTimeSeries')
            x_time = np.array(haemo_multi.get('time'))

            hbo_dataset = np.array(x_data[:, ::2])

            stim1 = np.array(f['nirs/stim1/data'])[:,0]
            stim3 = np.array(f['nirs/stim2/data'])[:,0]


        hbo = pd.DataFrame(hbo_dataset)
        stim1 = pd.DataFrame(stim1)
        stim3 = pd.DataFrame(stim3)

        # sets hbo dataFrame column names to match their channel and adds a column for time
        hbo.rename(columns={old_col: new_col for old_col, new_col in zip(hbo.columns, hbo_cols)}, inplace=True)
        hbo.insert(0, 'time', x_time)

        # creates a single dataframe to hold all stimulus values
        stim1["id"] = 1
        stim3["id"] = 0
        stim1.rename(columns={stim1.columns[0]: "time"}, inplace=True)
        stim3.rename(columns={stim3.columns[0]: "time"}, inplace=True)
        stims = pd.concat([stim1, stim3]).sort_values(by='time')

        # drops all non event related data
        hbo['id'] = hbo['time'].apply(get_id)
        hbo.insert(1, 'id', hbo.pop('id'))
        hbo.dropna(inplace=True)
        
        # sets is_event column to contain whether or not this is the exact time that stimulus was provided
        hbo['is_event'] = 0
        for stim_time in stims['time']:
            min_diff = np.abs(hbo['time'] - stim_time).idxmin()
            hbo.loc[min_diff, 'is_event'] = 1
        hbo.insert(2, 'is_event', hbo.pop('is_event'))

        target = stims

        event_indices = hbo[hbo['is_event'] == 1].index
        hbo.insert(2, 'event_index', 0)

        # Set the 'event_index' value to 1 for the rows 5 seconds before 'is_event' equals 1
        for index in event_indices:
            time_of_event = hbo.loc[index, 'time']
            time_before = time_of_event - 5
            if hbo.empty:
                hbo.loc[hbo['time'] >= time_before, 'event_index'] += 1
                # print('should not have ran')
            else:
                hbo.loc[hbo['time'] >= time_before, 'event_index'] += 1
                # hbo.loc[hbo['time'] >= time_before, 'event_index'] += 1

        target = hbo[["event_index","id"]]
        
        hbo['id'] = hbo['id'].astype(int)
        target['id'] = target['id'].astype(int)

        hbo_by_event = hbo.copy()
        # target.drop_duplicates(subset=['event_index'], inplace=True)    
        hbo_by_event.drop_duplicates(subset=['event_index'], inplace=True)

        target = target.set_index('event_index')
        # hbo = hbo.set_index('event_index')
        hbo_by_event = hbo_by_event.set_index('event_index')


        target = pd.Series(data=target["id"], index=target.index)
        hbo = hbo.drop(columns = ['id','time'])
        

        # display(target.head())
        # print(f"\ntarget shape: {target.shape}")

        # display(hbo.tail())
        # print(f"\nhbo shape: {hbo.shape}")

        display(hbo_by_event.head())
        print(f"\nhbo_by_event shape: {hbo_by_event.shape}")

        hbo = pd.concat([hbo,hbo], ignore_index=True)
        target_all = pd.concat([target,target],ignore_index=True)
        

        # display(hbo.tail())
        # print(f"hbo shape: {hbo.shape}")

        # display(target_all.head())
        # print(f"target_all shape: {target_all.shape}")

# target_all = pd.Series(data=target_all["id"], index=target_all.index)
print(f"target_all shape: {target_all.shape}")        

Loading C:\Users\dalto\Downloads\project\sourcedata_stim\sub-01\ses-01\nirs\sub-01_ses-01_nirs.snirf


C:\Users\dalto\AppData\Local\Temp\ipykernel_20036\1699478881.py:45: RuntimeWarning: Did not find any events.tsv associated with sub-01_ses-01.

The search_str was "C:\Users\dalto\Downloads\project\sourcedata_stim\sub-01\**\nirs\sub-01_ses-01*events.tsv"
  raw_haemo = read_raw_bids(raw_path)
C:\Users\dalto\AppData\Local\Temp\ipykernel_20036\1699478881.py:45: RuntimeWarning: Did not find any channels.tsv associated with sub-01_ses-01.

The search_str was "C:\Users\dalto\Downloads\project\sourcedata_stim\sub-01\**\nirs\sub-01_ses-01*channels.tsv"
  raw_haemo = read_raw_bids(raw_path)
C:\Users\dalto\AppData\Local\Temp\ipykernel_20036\1699478881.py:45: RuntimeWarning: Did not find any nirs.json associated with sub-01_ses-01.

The search_str was "C:\Users\dalto\Downloads\project\sourcedata_stim\sub-01\**\nirs\sub-01_ses-01*nirs.json"
  raw_haemo = read_raw_bids(raw_path)
C:\Users\dalto\AppData\Local\Temp\ipykernel_20036\1699478881.py:45: RuntimeWarning: participants.tsv file not found for C:

raw.info: 43

x available lists: ['dataTimeSeries', 'measurementList1', 'measurementList10', 'measurementList11', 'measurementList12', 'measurementList13', 'measurementList14', 'measurementList15', 'measurementList16', 'measurementList17', 'measurementList18', 'measurementList19', 'measurementList2', 'measurementList20', 'measurementList21', 'measurementList22', 'measurementList23', 'measurementList24', 'measurementList25', 'measurementList26', 'measurementList27', 'measurementList28', 'measurementList29', 'measurementList3', 'measurementList30', 'measurementList31', 'measurementList32', 'measurementList33', 'measurementList34', 'measurementList35', 'measurementList36', 'measurementList37', 'measurementList38', 'measurementList39', 'measurementList4', 'measurementList40', 'measurementList41', 'measurementList42', 'measurementList43', 'measurementList44', 'measurementList45', 'measurementList46', 'measurementList47', 'measurementList48', 'measurementList49', 'measurementList5', 'measure

C:\Users\dalto\AppData\Local\Temp\ipykernel_20036\1699478881.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['id'] = target['id'].astype(int)


,time,id,is_event,S1_D1 760,S1_D16 760,S2_D3 760,S3_D10 760,S4_D3 760,S4_D17 760,S5_D5 760,...,S9_D8 850,S9_D20 850,S10_D10 850,S11_D11 850,S12_D12 850,S13_D12 850,S13_D14 850,S13_D22 850,S14_D15 850,S15_D15 850
event_index,,,,,,,,,,,,,,,,,,,,,
1,50.50368,0,0,0.005579,0.326369,0.014329,0.209916,0.001204,0.251827,0.110648,...,0.007882,0.641227,0.077501,0.029581,0.057954,0.051245,0.012981,0.600144,0.056344,0.211949
2,82.20672,0,0,0.005399,0.325262,0.013784,0.208943,0.001195,0.252196,0.107675,...,0.007863,0.638065,0.076755,0.030328,0.059199,0.051477,0.013225,0.596289,0.056192,0.214306
3,108.93312,1,0,0.005575,0.327725,0.014381,0.212541,0.001231,0.250746,0.106188,...,0.007956,0.642397,0.077172,0.030271,0.059657,0.052366,0.013334,0.592969,0.058841,0.219273
4,141.00480,1,0,0.005452,0.324756,0.013870,0.208370,0.001197,0.250562,0.102312,...,0.007755,0.632151,0.075157,0.029914,0.058589,0.051375,0.013085,0.585626,0.056185,0.212025
5,184.32000,0,0,0.005458,0.324324,0.014248,0.208847,0.001245,0.251236,0.109453,...,0.007643,0.633200,0.075513,0.029983,0.060188,0.052528,0.013348,0.587346,0.058945,0.212377



hbo_by_event shape: (24, 46)
target_all shape: (8166,)


In [22]:
for sub in range(1,2):
        # hbo = pd.DataFrame()
        # target = pd.Series()
        
        subject_id = "%02d" % sub
        raw_path = BIDSPath(
            subject="%02d" % sub,
            task="task",
            # session='01',
            datatype="nirs",
            suffix='nirs',
            root=r"C:\Users\dalto\Downloads\project\sourcedata_lm",
            extension=".snirf"
        )

        # events, event_id = mne.events_from_annotations(mne.io.read_raw_snirf(raw_path))
        # raw_haemo, epochs = individual_analysis(raw_path)

        # print(f"epochs: \n{epochs.events[37:, [0, 2]]}\n")
        # print(f"epochs size: {raw_haemo.info}\n")

        raw_haemo = read_raw_bids(raw_path)

        hbo_cols = raw_haemo.info.ch_names[::2]
        print(f"raw.info: {len(hbo_cols)}\n")
        

        with h.File(raw_path,'r') as f:
            haemo_multi = f['nirs/data1']
            x_list = list(haemo_multi)
            print(f"x available lists: {x_list}\n")

            x_data = haemo_multi.get('dataTimeSeries')
            x_time = np.array(haemo_multi.get('time'))
            # print(f"x time: \n{x_time}\n")

            hbo_dataset = np.array(x_data[:, ::2])

            stim1 = np.array(f['nirs/stim1/data'])[:,0]
            stim3 = np.array(f['nirs/stim3/data'])[:,0]

Loading C:\Users\dalto\Downloads\project\sourcedata_lm\sub-01\nirs\sub-01_task-task_nirs.snirf


Reading events from C:\Users\dalto\Downloads\project\sourcedata_lm\sub-01\nirs\sub-01_task-task_events.tsv.
Reading channel info from C:\Users\dalto\Downloads\project\sourcedata_lm\sub-01\nirs\sub-01_task-task_channels.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
raw.info: 46

x available lists: ['dataTimeSeries', 'measurementList1', 'measurementList10', 'measurementList11', 'measurementList12', 'measurementList13', 'measurementList14', 'measurementList15', 'measurementList16', 'measurementList17', 'measurementList18', 'measurementList19', 'measurementList2', 'measurementList20', 'measurementList21', 'measurementList22', 'measurementList23', 'measurementList24', 'measurementList25', 'measurementList26', 'measurementList27', 'measurementList28', 'measurementList29', 'measurementList3', 'measurementList30', 'measurementList31', 'measurementList32', 'measurementList33', 'measurementList34', 'measurementList35', 'measurementList36', 'measurementList37', 'measurement

Feature Exrtraction

In [23]:
@set_property("fctype", "simple")

def evoked(x):
    """
    Distance traveled in an axis

    :param x: the time series to calculate the feature of
    :type x: pandas.Series
    :return: the value of this feature
    :return type: bool, int or float
    """
    # Distance traveled in one axis
    result = x[-1] - x[0]
    return result

In [24]:
fc_parameters = {
                 'abs_energy':None, # Returns the absolute energy of the time series which is the sum over the squared values
                 'absolute_sum_of_changes':None, # Returns the sum over the absolute value of consecutive changes in the series x
                 'fft_aggregated':[ {'aggtype': 'centroid'}, # Returns the spectral centroid (mean), 
                                    {'aggtype': 'variance'}, # variance, 
                                    {'aggtype': 'skew'},     # skew, 
                                    {'aggtype': 'kurtosis'}], # and kurtosis of the absolute fourier transform spectrum.
                 'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}], # Uses c3 statistics to measure non linearity in the time series
                 'standard_deviation': None, # Returns the standard deviation of x
                 'variance': None, # Returns the variance of x
                 'skewness': None, # Returns the sample skewness of x (calculated with the adjusted Fisher-Pearson standardized moment coefficient G1).
                 'kurtosis': None, # Returns the kurtosis of x (calculated with the adjusted Fisher-Pearson standardized moment coefficient G2).
                 'maximum': None, # Calculates the highest value of the time series x.
                 'minimum': None, # Calculates the lowest value of the time series x.
                 'sample_entropy':None, # Calculate and return sample entropy of x.
                 'mean_abs_change':None, # Average over first differences.
                 'sum_values':None, # Calculates the sum over the time series values
                #  'evoked': None # Calculates the evoked amplitude of the waveform
                # 'id' : 'id'
                }

feature_calculators.__dict__["evoked"] = evoked

Extract Features Using tsfresh

In [25]:
# x = extract_features(hbo, 
#                      column_id="event_index", 
#                      column_sort="time", 
#                      default_fc_parameters=fc_parameters, 
#                      impute_function=impute)

In [26]:
# print(f"\nx.shape: {x.shape}")
# x.tail()

In [27]:
# x_train, x_test, y_train, y_test = train_test_split(x, target, test_size=0.2, random_state=1)

random_state_list = [42]
n_splits = 10


def split_data(X, y, random_state_list):
        for random_state in random_state_list:
            kf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
            for train_index, val_index in kf.split(X, y):
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                yield X_train, X_val, y_train, y_val

split_data(hbo,target_all,random_state_list)

# x_train = x_train.sort_index()

# x_train['id'] = hbo_by_event['id']
# x_test['id'] = hbo_by_event['id']

# x_train.head()

<generator object split_data at 0x000001FEC84414D0>

In [28]:
# y_train = y_train.sort_index()

# y_train.head()

In [29]:
# features_filtered_direct = select_features(x_train, 
#                                            y_train,
#                                            ml_task="classification", # Classification o regression, by default set to auto
#                                            fdr_level = 0 # Respected percentage of features to be discarded as irrelevant, by default set to 0.05
#                                           )

In [30]:
# features_filtered_direct.head()

In [31]:
# DTC = DecisionTreeClassifier()
# DTC.fit(x_train, y_train)

In [32]:
# KNN = KNeighborsClassifier(n_neighbors=21)

# KNN.fit(x_train,y_train)

# KNN.score(x_test,y_test)

NameError: name 'x_train' is not defined

In [33]:
scores = []

for i, (xtrain, x_val, ytrain, y_val) in enumerate(split_data(hbo, target_all, random_state_list=random_state_list)):
    n = i % n_splits
    m = i // n_splits
    model = KNeighborsClassifier(n_neighbors=50)
    model.fit(xtrain, ytrain)
    score = model.score(x_val, y_val)
    scores.append(score)
    print(score)

print(f"\naverage scores: {np.average(scores)}")

0.9706242350061199
0.9596083231334149
0.9681762545899633
0.9632802937576499
0.9681762545899633
0.9596083231334149
0.9681372549019608
0.9705882352941176
0.9607843137254902
0.9705882352941176

average scores: 0.9659571723426212


In [34]:
scores = []

for i, (xtrain, x_val, ytrain, y_val) in enumerate(split_data(hbo, target_all, random_state_list=random_state_list)):
    n = i % n_splits
    m = i // n_splits
    model = SVC()
    model.fit(xtrain, ytrain)
    score = model.score(x_val, y_val)
    scores.append(score)
    print(score)

print(f"\naverage scores: {np.average(scores)}")

0.631578947368421
0.6144430844553244
0.5936352509179926
0.6119951040391677
0.6107711138310894
0.602203182374541
0.6115196078431373
0.6066176470588235
0.625
0.6053921568627451

average scores: 0.6113156094751242


In [35]:
scores = []

for i, (xtrain, x_val, ytrain, y_val) in enumerate(split_data(hbo, target_all, random_state_list=random_state_list)):
    n = i % n_splits
    m = i // n_splits
    model = xgb.XGBClassifier()
    model.fit(xtrain, ytrain)
    score = model.score(x_val, y_val)
    scores.append(score)
    print(score)

print(f"\naverage scores: {np.average(scores)}")

1.0
0.9975520195838433
0.9951040391676866
0.9975520195838433
1.0
1.0
0.9975490196078431
1.0
1.0
1.0

average scores: 0.9987757097943216


In [ ]:
print((y_test, KNN.predict(x_test)))

NameError: name 'y_test' is not defined